# Memory

Describing memory and getting it synthesized correcly is cumbersome.
Therefore, this will only demonstrate a simple approach to generate memory using the intermediate representation language dialect (IRL).
This modification enables shared variable output for VHDL to support true dual port behaviour.
The general strategy with the MyIRL synthesis is to use a blackbox element (rely on the library)

In [1]:
from myhdl import intbv
from myirl import *
import random

## Dual port memories

We can not use a `SigArray` for the RAM, because a true dual port RAM would provoke unresolved multiple drivers.
Therefore we hack the `myirl.lists.SigArray` class by deriving and choosing a different element in return of the `__getitem__` member which uses a shared variable.

**Note** Not working in VHDL-2008

In [2]:
from myirl import lists
from myirl.kernel import extensions, utils

from myirl import targets

class CellAssign:
    def __init__(self, parent, portion, val, width):
        self._vref = parent
        self.portion = portion
        self.value = val
        self.width = width
        
    def emit(self, context):
        tgt = context.target
        if tgt.lang != 'VHDL':
            raise TypeError("Unsupported target %s" % type(tgt))
        n, v = self._vref.identifier, self.value
        sz = self.width
        ps = self.portion.convert(tgt)
        context.output("%s(to_integer(%s)) := %s;\n" % (n, ps, base.convert(v, tgt, sz)))        

class ArrayElem(lists.SigIndexed):
    decl_type_vhdl = "shared variable"
    
    def set(self, val):
        w = self.size()
        return CellAssign(self.seq, self.index, val, w)

class RamBuffer(lists.SigArray):    
    def __getitem__(self, item):
        if isinstance(item, (Sig, int)):
            # We can not just return self.val[item], as the iterator
            # has not initialized yet.
            return ArrayElem(self, item)
        else:
            raise TypeError("Multi item slicing of iterator not supported")

In [3]:
from myirl.library.bulksignals import bulkwrapper

@bulkwrapper(targets.vhdl, TYPE='inout')
class RamPort:
    _inputs = ['we', 'wa', 'ra', 'wd']
    _outputs = ['rd']
    _other = ['clk']
    def __init__(self, AWIDTH, DWIDTH):
        self.clk = ClkSignal()
        self.we = Signal(bool())
        self.ra, self.wa = [ Signal(intbv()[AWIDTH:]) for i in range(2) ]
        self.rd, self.wd = [ Signal(intbv()[DWIDTH:]) for i in range(2) ] 
        
@block
def tdp_ram(pa, pb, INITDATA):
    inst = []
 
    def gen_logic(p, i):
        "Generate port mechanics inline"
        @genprocess(p.clk, EDGE=p.clk.POS)
        def proc():
            yield [
                proc.If(p.we == True).Then(
                    buf[p.wa].set(p.wd)
                ),
                p.rd.set(buf[p.ra])
            ]
        proc.rename("proc%d" % i)
        return proc

    buf = RamBuffer(INITDATA)
    
    for i, p in enumerate([pa, pb]):
        inst.append(gen_logic(p, i))
    
    return inst

In [4]:
@block
def tb(RAM_CONTENT):
    pa, pb = [ RamPort(AWIDTH=9, DWIDTH=8, name=n) for n in ['A', 'B'] ]
    inst = tdp_ram(pa, pb, RAM_CONTENT)

    return instances()

In [5]:
from myirl import targets
from myirl.test.common_test import run_ghdl

random.seed(0)

def test():
    RAM_CONTENT = [ intbv(random.randint(0, 2 ** 9))[8:] for i in range(2 ** 9) ]
    inst = tb(RAM_CONTENT)
    
    f = inst.elab(targets.VHDL, elab_all = True)
    run_ghdl(f, inst, std = "93", debug = True) # Note we run with std '93'

In [6]:
test()

VHDL target: REGISTERING `RamPort_9_8_in` <class 'myirl.library.bulksignals.RamPort_9_8_in'>
VHDL target: REGISTERING `RamPort_9_8_out` <class 'myirl.library.bulksignals.RamPort_9_8_out'>
VHDL target: REGISTERING `RamPort_9_8_aux` <class 'myirl.library.bulksignals.RamPort_9_8_aux'>
 Writing 'tdp_ram' to file /tmp/myirlee9itc_u/tdp_ram.vhdl 
Finished _elab in 0.0009 secs
 Writing 'tb' to file /tmp/myirlee9itc_u/tb.vhdl 
Finished _elab in 0.0007 secs
 Creating library file /tmp/myirluqw25xdi/module_defs.vhdl 
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /home/testing/.local/lib/python3.10/site-packages/myirl-0.0.0-py3.10-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/testing/.local/lib/python3.10/site-packages/myirl-0.0.0-py3.10-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/myirluqw25xdi/module_defs.vhdl
analyze /tmp/myirlee9itc_u/tdp_ram.vhdl
analyze /tmp/myirlee9itc_u/tb.vhdl
elaborate tb

==== COSIM stderr ====

==

In [7]:
! cat -n {tdp_ram.ctx.path_prefix}/tdp_ram.vhdl 

     1	-- File generated from /usr/local/lib/python3.10/runpy.py
     2	-- (c) 2016-2021 section5.ch
     3	-- Modifications may be lost
     4	
     5	library IEEE;
     6	use IEEE.std_logic_1164.all;
     7	use IEEE.numeric_std.all;
     8	
     9	library work;
    10	
    11	use work.module_defs.all;
    12	use work.txt_util.all;
    13	use work.myirl_conversion.all;
    14	
    15	entity tdp_ram is
    16	    port (
    17	        pa_bulk0682_in : in t_RamPort_9_8_in;
    18	        pa_bulk0682_out : out t_RamPort_9_8_out;
    19	        pa_bulk0682_aux : in t_RamPort_9_8_aux;
    20	        pb_bulk04c3_in : in t_RamPort_9_8_in;
    21	        pb_bulk04c3_out : out t_RamPort_9_8_out;
    22	        pb_bulk04c3_aux : in t_RamPort_9_8_aux
    23	    );
    24	end entity tdp_ram;
    25	
    26	architecture MyIRL of tdp_ram is
    27	    -- Local type declarations
    28	    -- Signal declarations
    29	    type a_s_3104 is array (0 to 511) of unsigned(7 downto 0);
    30	    shared 